# Tensors

### Gradients as equation solver

In [ ]:
import torch
import matplotlib.pyplot as plt

<!-- $((x - 0.2)^{2} - 1)^{2} + \frac{1}{3}$ -->

$f(x) = x^4 - 0.7x^3 - 2x^2 + x + 1$

In [ ]:
def peaks(x):
  return x**4 - 0.7*x**3 - 2*x**2 + x + 1

In [ ]:
x = torch.linspace(-1.3, 1.6, 300)
y = peaks(x)

plt.plot(x, y)
plt.show()

In [ ]:
xs = []
ys = []

x_ = torch.tensor(0.5, requires_grad=True)

for e in range(10):
  y_ = peaks(x_)
  xs.append(x_.item())
  ys.append(y_.item())

  y_.backward()
  print(x_, y_, x_.grad)

  with torch.no_grad():
    x_ -= 0.1 * x_.grad
    x_.grad = None


In [ ]:
plt.plot(x, y)
plt.scatter(xs, ys, marker='o', s=14, c='r')
plt.show()
x_.item(), y_.item()

### Gradients as number finders

In [ ]:
import matplotlib.pyplot as plt
import torch

In [ ]:
target = torch.zeros((3, 3)).uniform_(-3, 3)

v = torch.randint(-4, 4, (3,3), dtype=torch.float, requires_grad=True)

target, v

In [ ]:
for i in range(16):
  diff = (v - target).pow(2).sum()
  diff.backward()

  print(diff.item())

  with torch.no_grad():
    v -= 0.1 * v.grad
    v.grad = None

target, v

### Gradients as Linear Regression

to approximate $sin(t)$:

$f(t) = w_A \cdot t + w_B \cdot t^2 + w_C \cdot t^3 + w_D \cdot t^5$

In [ ]:
t = torch.linspace(-torch.pi, torch.pi, 24)
ft = torch.sin(t)

w = torch.zeros((1, 4), requires_grad=True)
fs = torch.stack([t, t**2, t**3, t**5])

print(w)

In [ ]:
for i in range(5000):
  ft_p = w.matmul(fs)
  loss = (ft_p - ft).pow(2).sum()
  loss.backward()

  with torch.no_grad():
    w -= 1e-6 * w.grad
    w.grad = None

  if i%100 == 0:
    print(i, loss.item())

loss.item(), w

In [ ]:
plt.scatter(t, ft, marker='o', s=10)
plt.plot(t, ft_p.detach().reshape(-1), color='r')
plt.show()

## Compare to Linear Regression

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

from data_utils import object_from_json_url, StandardScaler, LinearRegression, PolynomialFeatures

In [ ]:
# Define the location of the json file here
HOUSES_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024F-H/9103-utils/main/datasets/json/LA_housing.json"

houses_info = object_from_json_url(HOUSES_FILE)

houses_raw_df = pd.DataFrame.from_records(houses_info)

std_scaler = StandardScaler()
houses_df = std_scaler.fit_transform(houses_raw_df)

houses_train, houses_test = train_test_split(houses_df, test_size=0.2)

houses_train.head()

In [ ]:
# Get prices and features
Y_train = houses_train["value"]
X_train = houses_train.drop(columns=["value"])

Y_test = houses_test["value"]
X_test = houses_test.drop(columns=["value"])

# Augment
# poly = PolynomialFeatures(degree=4, include_bias=False)
X_train_poly = X_train
X_test_poly = X_test

# Linear Regression
price_model = LinearRegression()

# Build the model
price_model.fit(X_train_poly, Y_train)

# Run the model on the datasets
predicted_std_train = price_model.predict(X_train_poly)
predicted_std_test = price_model.predict(X_test_poly)

# Un-normalize the data
predicted_train = std_scaler.inverse_transform(predicted_std_train)
predicted_test = std_scaler.inverse_transform(predicted_std_test)

# Measure Model error
rmse_train = root_mean_squared_error(Y_train.values, predicted_std_train.values)
rmse_test = root_mean_squared_error(Y_test.values, predicted_std_test.values)

print("train error:", rmse_train, "\ntest error:", rmse_test)


In [ ]:
# Get prices and features
Y_train = houses_train["value"]
X_train = houses_train.drop(columns=["value"])

Y_test = houses_test["value"]
X_test = houses_test.drop(columns=["value"])

# Augment
poly = PolynomialFeatures(degree=4, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Linear Regression
price_model = LinearRegression()

# Build the model
price_model.fit(X_train_poly, Y_train)

# Run the model on the datasets
predicted_std_train = price_model.predict(X_train_poly)
predicted_std_test = price_model.predict(X_test_poly)

# Un-normalize the data
predicted_train = std_scaler.inverse_transform(predicted_std_train)
predicted_test = std_scaler.inverse_transform(predicted_std_test)

# Measure Model error
rmse_train = root_mean_squared_error(Y_train.values, predicted_std_train.values)
rmse_test = root_mean_squared_error(Y_test.values, predicted_std_test.values)

print("train error:", rmse_train, "\ntest error:", rmse_test)
